### Задание 1.

- Посмотреть документацию к API GitHub,
- разобраться как вывести список репозиториев для конкретного пользователя,
- сохранить JSON-вывод в файле *.json.

In [1]:
# REST    API v3 - https://developer.github.com/v3/
# GraphQL API v4 - https://developer.github.com/v4/

In [2]:
import requests
from getpass import getpass
import json

#### Настройки

In [3]:
GIT_API_URL='https://api.github.com'
REST_MASK__INFO_ABOUT_THE_USER_REPOS = '/users/#username#/repos'

In [4]:
# Заголовки Google Chrome
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
                  ' AppleWebKit/537.36 (KHTML, like Gecko)'
                  ' Chrome/83.0.4103.106 Safari/537.36',
    'Accept': 'application/vnd.github.v3+json' 
    # 'Accept': '*/*'
        }

In [5]:
# Параметры запроса
params = {
    'type': 'owner',
    'sort': 'created',
    'direction': 'desc'
}

In [6]:
# Интересуемый субъект
target_github_users = ['Unexisting_user', 'axeldawya', 'test','ivanov']

In [7]:
# Авторизация

github_u, github_p = '', ''
# github_u, github_p = input("GitHub login:"), getpass(prompt="GitHub password:")  

the_auth = None
if github_u != '':
    the_auth = (github_u, github_p)

#### Вспомогательные функции

In [8]:
def request_info(p_the_URL, p_headers, p_params, p_auth):

    return_data = {}
    response = requests.get(p_the_URL, headers=p_headers, params=p_params, auth=p_auth)
    #print(f'...response.url         = {response.url}')
    #print(f'...response.status_code = {response.status_code}')
    #print(f'...response.ok          = {response.ok}')
    #print(f'...response.text        = {response.text}')
    #print(f'...response.json()      = {response.json()}')

    if response.ok:
        if response.status_code == 200:
            return_data = response.json()
    else:
        if response.status_code == 404:
            if response.json().get('message') == 'Not Found':
                return_data = None

    return return_data

In [9]:
def show_info(repos):
    if repos is None:
        print('  User is not found')
    else:
        if len(repos) == 0:
            print('  User has no specified public repositories to show')
        else:
            for i_repo in repos:
                if not i_repo['private']:
                    print('-', i_repo['html_url'])

In [12]:
def save_info(p_git_user, p_json_data):
    json_filename = f'github_repositories_of_user_{p_git_user}.json'
    
    try:
        with open(json_filename, 'w', encoding='utf-8') as the_file:
            json.dump(p_json_data, the_file, ensure_ascii=False, indent=4)
    except IOError:
        print(f'  Внимание! Не удалось сохранить данные в файл "{json_filename}"!')
    else:
        print(f'  Данные записаны в файл "{json_filename}"')

#### Зрелище

In [13]:
for i_target_user in target_github_users:
    
    # prepare
    i_URL = GIT_API_URL + REST_MASK__INFO_ABOUT_THE_USER_REPOS.replace("#username#", i_target_user)
    print(f'User: {i_target_user}')
    
    # request for user repositories
    repos = request_info(i_URL, HEADERS, params, the_auth)
    
    # show results
    show_info(repos)
                    
    # save data to files
    if repos:
        save_info(i_target_user, repos)

User: Unexisting_user
  User is not found
User: axeldawya
  User has no specified public repositories to show
User: test
- https://github.com/test/HelloWorld
- https://github.com/test/SDWebImage
- https://github.com/test/Test--01
- https://github.com/test/rokehan
- https://github.com/test/sNews
  Данные записаны в файл "github_repositories_of_user_test.json"
User: ivanov
- https://github.com/ivanov/jlsp-test-drive
- https://github.com/ivanov/pep582
- https://github.com/ivanov/asterisk-blog
- https://github.com/ivanov/pipx
- https://github.com/ivanov/lz4
- https://github.com/ivanov/summit
- https://github.com/ivanov/jothon-net
- https://github.com/ivanov/matplotlib-wheels
- https://github.com/ivanov/voila
- https://github.com/ivanov/jupyter_client
- https://github.com/ivanov/ipynb
- https://github.com/ivanov/cellout
- https://github.com/ivanov/husky
- https://github.com/ivanov/potatoes
- https://github.com/ivanov/global-sprint
- https://github.com/ivanov/scientific-python-cookiecutter
-

#### Текущие недостатки решения
По умолчанию действует ограничение на 30 записей. Поэтому нужно еще поработать над получением полной выборки